In [3]:
from math import sqrt
from operator import itemgetter

import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn import metrics
from MF import PureSingularValueDecomposition

In [4]:
class RegularizedSingularValueDecomposition:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        self.n = len(np.unique(np.sort(records[:, 0])))
        self.m = len(np.unique(np.sort(records[:, 1])))

        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=np.int32)

        for record in records_train:
            self.R[record[0], record[1]] = record[2]

        # Initial indicator
        y = np.where(self.R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)

        # Global average of rating
        self.r = np.sum(self.R) / np.sum(y)

        # average rating of user
        self.r_u = np.where(y_user,
                            np.sum(self.R, axis=1) / y_user,
                            self.r)

        # average rating of item
        self.r_i = np.where(y_item,
                            np.sum(self.R, axis=0) / y_item,
                            self.r)

        # bias of user
        self.b_u = np.where(y_user,
                            np.sum(y * (self.R - self.r_i), axis=1) / y_user,
                            0)

        # bias of item
        self.b_i = np.where(y_item,
                            np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
                            0)
        
    def gradient_descent(self, n_iter=5):

        alpha = 0.01
        d = 20
        
        # Initialize
        U = (np.random.randint(0, 1, size=(self.n, d)) - 0.5) * 0.01
        V = (np.random.randint(0, 1, size=(self.m, d)) - 0.5) * 0.01
        mu = self.r
        b_i = self.b_i
        b_u = self.b_u
        
        eta = 0.01
        
        def dJ_sgd(mu, b_u, b_i, U, V, r):
            e = r - (mu + b_u + b_i + U.dot(V))
            return -e, -e + alpha * b_u, -e + alpha * b_i, -e * V + alpha * U, -e * U + alpha * V
        
        for cur_iter in range(n_iter):
            print(cur_iter)
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]

            for i in range(num):
                user = users[i]
                item = items[i]
                gradient_mu, gradient_b_u, gradient_b_i, gradient_U, gradient_V = dJ_sgd(mu, b_u[user], b_i[item], U[user, :], V[item, :], self.R[user, item])
                mu -= eta * gradient_mu
                b_u[user] -= eta * gradient_b_u
                b_i[item] -= eta * gradient_b_i
                U[user, :] -= eta * gradient_U
                V[item, :] -= eta * gradient_V
                
                # b_u -= eta * gradient_b_u
                # b_i -= eta * gradient_b_i
                # U -= eta * gradient_U
                # V -= eta * gradient_V
                
            eta = eta * 0.9
            ratings_predict_rsvd = performance(mu, b_u, b_i, U, V, records_test)
            print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))

        return mu, b_u, b_i, U, V

In [5]:
class Nomu:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        self.n = len(np.unique(np.sort(records[:, 0])))
        self.m = len(np.unique(np.sort(records[:, 1])))

        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=np.int32)

        for record in records_train:
            self.R[record[0], record[1]] = record[2]

        # Initial indicator
        y = np.where(self.R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)

        # Global average of rating
        self.r = np.sum(self.R) / np.sum(y)

        # average rating of user
        self.r_u = np.where(y_user,
                            np.sum(self.R, axis=1) / y_user,
                            self.r)

        # average rating of item
        self.r_i = np.where(y_item,
                            np.sum(self.R, axis=0) / y_item,
                            self.r)

        # bias of user
        self.b_u = np.where(y_user,
                            np.sum(y * (self.R - self.r_i), axis=1) / y_user,
                            0)

        # bias of item
        self.b_i = np.where(y_item,
                            np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
                            0)
        
    def gradient_descent(self, n_iter=5):

        alpha = 0.01
        d = 20
        
        # Initialize
        U = (np.random.randint(0, 1, size=(self.n, d)) - 0.5) * 0.01
        V = (np.random.randint(0, 1, size=(self.m, d)) - 0.5) * 0.01
        mu = self.r
        b_i = self.b_i
        b_u = self.b_u
        
        eta = 0.01
        
        def dJ_sgd(mu, b_u, b_i, U, V, r):
            e = r - (mu + b_u + b_i + U.dot(V))
            return -e, -e + alpha * b_u, -e + alpha * b_i, -e * V + alpha * U, -e * U + alpha * V
        
        for cur_iter in range(n_iter):
            print(cur_iter)
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]

            for i in range(num):
                user = users[i]
                item = items[i]
                gradient_mu, gradient_b_u, gradient_b_i, gradient_U, gradient_V = dJ_sgd(mu, b_u[user], b_i[item], U[user, :], V[item, :], self.R[user, item])
                # mu -= eta * gradient_mu
                b_u[user] -= eta * gradient_b_u
                b_i[item] -= eta * gradient_b_i
                U[user, :] -= eta * gradient_U
                V[item, :] -= eta * gradient_V
                
                # b_u -= eta * gradient_b_u
                # b_i -= eta * gradient_b_i
                # U -= eta * gradient_U
                # V -= eta * gradient_V
                
            eta = eta * 0.9
            ratings_predict_rsvd = performance(mu, b_u, b_i, U, V, records_test)
            print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))

        return mu, b_u, b_i, U, V

In [10]:
class MatrixFactorization:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        self.n = len(np.unique(np.sort(records[:, 0])))
        self.m = len(np.unique(np.sort(records[:, 1])))

        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=np.int32)

        for record in records_train:
            self.R[record[0], record[1]] = record[2]

        # Initial indicator
        y = np.where(self.R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)

        # Global average of rating
        self.r = np.sum(self.R) / np.sum(y)

        # average rating of user
        self.r_u = np.where(y_user,
                            np.sum(self.R, axis=1) / y_user,
                            self.r)

        # average rating of item
        self.r_i = np.where(y_item,
                            np.sum(self.R, axis=0) / y_item,
                            self.r)

        # bias of user
        self.b_u = np.where(y_user,
                            np.sum(y * (self.R - self.r_i), axis=1) / y_user,
                            0)

        # bias of item
        self.b_i = np.where(y_item,
                            np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
                            0)
        
    def gradient_descent(self, n_iter=5000):


        R_svd = np.where(self.R == 0,
                          np.zeros(shape=self.R.shape) + self.r_u.reshape(-1, 1),
                          self.R)
        # SVD
        U, s, VT = svd(R_svd)

        d = 20
        Sigma = np.zeros([d, d])
        for i in range(d):
            Sigma[i][i] = s[i]
        
        U = U[:, :d].dot(Sigma)
        V = VT[:d, :].T
        
        def performance():
            return U.dot(V.T)[records_test[:, 0], records_test[:, 1]]
        ratings_predict_rsvd = performance()
        print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))
        print("GG")
        eta = 0.0001
        alpha = 0.0001
        mu = self.r
        
        def dJ_sgd(U, V, r):
            e = r - U.dot(V)
            return -e * V + alpha * U, -e * U + alpha * V

        for cur_iter in range(n_iter):
            print(cur_iter)
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]
            
            for i in range(num):
                user = users[i]
                item = items[i]
                gradient_U, gradient_V = dJ_sgd(U[user, :], V[item, :], self.R[user, item])
                
                U[user, :] -= eta * gradient_U
                V[item, :] -= eta * gradient_V
                ratings_predict_rsvd = performance()
                if not ratings_predict_rsvd.min() > -10:
                    print(i)
                    break
                # U -= eta * gradient_U
                # V -= eta * gradient_V
                
            eta = eta * 0.9
            ratings_predict_rsvd = performance()
            # print(ratings_predict_rsvd.min() > -10)
            # print(ratings_predict_rsvd[:20])
            print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))

        return

In [7]:
def score(ratings_test, ratings_predict):
    return [round(sqrt(metrics.mean_squared_error(ratings_test, ratings_predict)), 4),
            round(metrics.mean_absolute_error(ratings_test, ratings_predict), 4)]
def performance(mu, b_u, b_i, U, V, records_test):
        return mu + b_u[records_test[:, 0]] + b_i[records_test[:, 1]] + U.dot(V.T)[records_test[:, 0], records_test[:, 1]]

In [8]:
# Load the records
records_train = np.loadtxt('../data/ml-100k/u1.base', dtype=np.int32)
records_test = np.loadtxt('../data/ml-100k/u1.test', dtype=np.int32)

# Preprocess
records_train[:, :2] -= 1
records_test[:, :2] -= 1
ratings_test = records_test[:, 2]
records = np.vstack([records_train, records_test])

In [104]:
%%time
psvd = PureSingularValueDecomposition(records_train, records_test)

/Users/mac/Downloads/recommender-algorithm-implementation/Matrix Factorization/MF.py:39: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,


CPU times: user 3.93 s, sys: 451 ms, total: 4.38 s
Wall time: 889 ms


In [105]:
ratings_predict = psvd.performance(records_test)
# ratings_predict.max()

print(score(np.clip(ratings_predict, 1, 5), ratings_test))
# score(np.clip(ratings_predict, 1, 5), ratings_test)

[1.017, 0.8058]


In [88]:
%%time
rsvd = RegularizedSingularValueDecomposition(records_train, records_test)

mu, b_u, b_i, U, V = rsvd.gradient_descent(30)
ratings_predict_rsvd = performance(mu, b_u, b_i, U, V, records_test)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


0
[0.9606, 0.7522]
1
[0.9566, 0.7543]
2
[0.9568, 0.7542]
3
[0.9569, 0.7515]
4
[0.9576, 0.7589]
5
[0.9558, 0.7549]
6
[0.9563, 0.7507]
7
[0.9551, 0.7528]
8
[0.9559, 0.7507]
9
[0.9577, 0.7494]
10
[0.9556, 0.7551]
11


KeyboardInterrupt: 

In [73]:
score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test)

[1.9394, 1.5405]

In [11]:
%%time
mf = MatrixFactorization(records_train, records_test)

mf.gradient_descent(70)



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


[1.017, 0.8058]
GG
0
383
[1.2812, 0.9907]
1
0
[1.275, 0.9852]
2
0
[1.2751, 0.9853]
3
0
[1.276, 0.9863]
4
0
[1.2759, 0.986]
5
0
[1.276, 0.9861]
6
0
[1.2762, 0.9864]
7
0
[1.2762, 0.9865]
8
0
[1.2762, 0.9865]
9
0
[1.2762, 0.9866]
10
0
[1.2734, 0.9845]
11
0
[1.2736, 0.9847]
12
0
[1.2737, 0.9848]
13
0
[1.2735, 0.9844]
14
0
[1.2735, 0.9844]
15
0
[1.2735, 0.9845]
16
0
[1.2737, 0.9848]
17
0
[1.2732, 0.9845]
18
0
[1.2731, 0.9843]
19
0
[1.2728, 0.9839]
20
0
[1.2723, 0.9834]
21
0
[1.2723, 0.9834]
22
0
[1.2721, 0.9831]
23
0
[1.2721, 0.9832]
24
0
[1.2721, 0.9832]
25
0
[1.2722, 0.9834]
26
0
[1.2722, 0.9833]
27
0
[1.2718, 0.9831]
28
0
[1.2717, 0.9829]
29
0
[1.2717, 0.9829]
30
0
[1.2717, 0.9829]
31
0
[1.2717, 0.9829]
32
0
[1.2717, 0.9829]
33
0
[1.2717, 0.9829]
34
0
[1.2716, 0.9829]
35
0
[1.2716, 0.9829]
36
0
[1.2716, 0.9829]
37
0
[1.2716, 0.9829]
38
0
[1.2716, 0.9828]
39
0
[1.2714, 0.9827]
40
0
[1.2714, 0.9827]
41
0
[1.2714, 0.9827]
42
0
[1.2714, 0.9826]
43
0
[1.2714, 0.9826]
44
0
[1.2714, 0.9827]
45


In [91]:
%%time
nomu = Nomu(records_train, records_test)
mu, b_u, b_i, U, V = nomu.gradient_descent(70)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


0
[0.9581, 0.7543]
1
[0.9568, 0.7536]
2
[0.9561, 0.7528]
3
[0.9562, 0.7526]
4
[0.9556, 0.752]
5
[0.9558, 0.7523]
6


KeyboardInterrupt: 